# Neuroevolution

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a>&nbsp;| Dennis G. Wilson | <a href="https://supaerodatascience.github.io/stochastic/">https://supaerodatascience.github.io/stochastic/</a>

This notebook was optimized for Google colab.

In [ ]:
!apt-get install -y xvfb python-opengl swig > /dev/null 2>&1

In [ ]:
!pip install swig > /dev/null 2>&1

In [ ]:
!pip install xvfbwrapper

In [ ]:
!pip install cma pyvirtualdisplay gymnasium[box2d]

Consider a task where sequential decisions must be made. A common model is to consider that an environment gives an observation for each decision, or action, taken. We can then make decisions by choosing a function which takes in an environment state and gives out an action.

<img src="https://raw.githubusercontent.com/SupaeroDataScience/stochastic/master/notebooks/imgs/ne_basics.png">

Evolutionary strategies are intended for continuous optimization and can easily be applied to the optimization of neural network parameters, or *neuroevolution*. The goal of this is to optimize the parameters of a function which can interact in an environment. A neural network can represent any function, so we can use it as a representation of our decision making function. The parameters of the neural network become the individual we are optimizing.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import numpy as np
import gymnasium

In [ ]:
class NeuralNetwork(nn.Module):

    def __init__(self, input_shape, n_actions):
        super(NeuralNetwork, self).__init__()
        self.l1 = nn.Linear(input_shape, 32)
        self.l2 = nn.Linear(32, 32)
        self.lout = nn.Linear(32, n_actions)

    def forward(self, x):
        x = F.relu(self.l1(x.float()))
        x = F.relu(self.l2(x))
        return self.lout(x)

    def get_params(self):
        p = np.empty((0,))
        for n in self.parameters():
            p = np.append(p, n.flatten().cpu().detach().numpy())
        return p

    def set_params(self, x):
        start = 0
        for p in self.parameters():
            e = start + np.prod(p.shape)
            p.data = torch.FloatTensor(x[start:e]).reshape(p.shape)
            start = e

We'll add some visualization functionality to have the environment render directly in the notebook. To run this notebook in Google colab, uncomment and run the following lines.

In [ ]:
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion();

Following the framework of evolutionary policy search, we will optimize a neural network representing a policy and maximize the total reward over a single episode using this policy. In the following function, we use the policy function to take actions and calculate a reward given by the environment.

In [ ]:
def evaluate(ann, env, visul=True):
    # set the environment
    obs, info = env.reset(seed=0, options={})
    # visualization
    if visul:
        img = plt.imshow(env.render())
    # total reward, objective for maximizing
    total_reward = 0
    # run the sequence of decisions
    done = False
    while not done:
        # Use the neural network to make a decision based on observations
        net_output = ann(torch.tensor(obs))
        # the action is the value clipped returned by the nn
        action = net_output.data.cpu().numpy().argmax()
        # take a step in the environment
        obs, reward, done, _, _ = env.step(action)
        # track the total reward
        total_reward += reward
        # visualization of the action
        if visul:
            img.set_data(env.render())
            plt.axis('off')
            display.display(plt.gcf())
            display.clear_output(wait=True)
    return total_reward

We've configured this for discrete action spaces. We can see a random neural network on different environments like `CartPole-v0`, `MountainCar-v0`, and `LunarLander-v2`.

In [ ]:
env = gymnasium.make("LunarLander-v2", render_mode="rgb_array")
ann = NeuralNetwork(env.observation_space.shape[0], env.action_space.n)

In [ ]:
evaluate(ann, env, visul=True)

In order to evolve the parameters of this neural network, we will modify the parameters of the network using `set_params` with the genes of the new individual. In the evolutionary literature, this is referred to as a *direct encoding* as the neural network parameters are directly encoded in the genome.

In [ ]:
def fitness(x, ann, env, visul=False):
    ann.set_params(x)
    return -evaluate(ann, env, visul=visul)

In [ ]:
p = ann.get_params()
np.shape(p)

We can first observe a random individual $x$.

In [ ]:
x = np.random.rand(len(p))
-fitness(x, ann, env, visul=True)

# CMA-ES for Neuroevolution

We will now use CMA-ES for the Lunar Lander problem using the pycma package (https://github.com/CMA-ES/pycma)

In [ ]:
import cma

In [ ]:
np.random.seed(123)
env = gymnasium.make('LunarLander-v2')
ann = NeuralNetwork(env.observation_space.shape[0], env.action_space.n)
es = cma.CMAEvolutionStrategy(len(ann.get_params()) * [0], 0.1, {'seed': 123})

In [ ]:
for i in range(20):
    solutions = np.array(es.ask())
    fits = [fitness(x, ann, env) for x in solutions]
    es.tell(solutions, fits)
    es.disp()

In [ ]:
x = es.result[0]
env = gymnasium.make('LunarLander-v2', render_mode="rgb_array")
-fitness(x, ann, env, visul=True)

The results on LunarLander clearly show the benefits of CMA-ES; we have found a reasonable policy in a small number of generations. Applying CMA-ES to larger neural networks remains an open challenge, however, due to the vast number of parameters in ANNs. Specifically, CMA-ES calculates the covariance of all parameters, which is $O(n^2)$.

In [ ]:
np.shape(es.sm.C)